In [1]:
import pandas as pd
import numpy as np
import re
from pathlib import Path
from collections import Counter
from typing import List, Dict, Tuple, Optional
import json

# NLP libraries
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import hdbscan  # Для кластеризации без фиксированного числа кластеров
import warnings
warnings.filterwarnings('ignore')

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")


/Users/a.danyarov/Library/Caches/pypoetry/virtualenvs/lamoda-bootcamp-9UBAvNEw-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
df_products.head()

,sku,category,product_type,product_name,reviews,num_reviews,clean_reviews,num_clean_reviews,review_embeddings,clusters,cluster_groups,raw_aspects,filtered_aspects,num_aspects
119186,RTLACN098001,Beauty_Accs,MAKE-UP EYES,Палетка для глаз,"[Отличные тени, мягкая текстура, хорошо ложатс...",1393,"[отличные тени, мягкая текстура, хорошо ложатс...",1393,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[-1, -1, 37, -1, 117, 147, 32, 30, -1, 171, -1...","{0: {'texts': ['покупаю не первый раз!', 'перв...","{0: ['раз заказала тени, все было', 'заказала ...","[супер качество по хорошей, отличное качество....",6
48815,MP002XW0FXPS,Clothes,SOCKS & TIGHTS,Носки 3 пары,"[По качеству хорошие, производитель Узбекистан...",1390,"[по качеству хорошие, производитель узбекистан...",1390,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[21, 3, -1, 176, 226, 127, 164, -1, 118, 91, -...","{0: {'texts': ['ожидала, что ткань будет плотн...","{0: ['ожидала, что ткань будет', 'что ткань бу...","[, на вид качество хорошее, качество не очень,...",6
56885,MP002XW0MX30,Beauty_Accs,EYES CARE,Патчи для глаз,"[Хорошие патчи , Самые любимые патчи! , Самые ...",1311,"[хорошие патчи, самые любимые патчи!, самые лю...",1311,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[188, 137, 39, -1, 149, 184, -1, 13, 114, 73, ...","{0: {'texts': ['хорошие патчи, только носить и...","{1: ['пожалуй лучшие патчи на', 'пожалуй лучши...","[эффекта «вау» не, хоть какой-то эффект, , осо...",6
41898,MP002XW00FJB,Beauty_Accs,MAKE-UP BROWS,Гель для бровей,"[Самый лучший гель! , Мне не оч «зашел» гель…(...",1292,"[самый лучший гель!, мне не оч «зашел» гель…(,...",1292,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[195, 33, 40, 163, 174, 21, -1, -1, -1, -1, 11...","{0: {'texts': ['много геля на щеточке, из-за э...","{3: ['понравился гель фиксирует идеально .', '...","[эффект ламинирования держится в, эффект ламин...",6
109922,MP002XW1FMAK,Clothes,UNDERWEAR,Трусы 4 шт.,"[Дополнение..\nНе очень понравилось, как сидят...",1100,"[дополнение.. не очень понравилось, как сидят ...",1100,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[83, 37, -1, -1, 203, 197, 157, 116, 87, 129, ...","{0: {'texts': ['скручиваются', 'звезду сняла ч...","{2: ['цены и качества', 'соотношение цены и'],...","[совсем поняла что с размерным, свой 42-44 взя...",6
37498,MP002XU08J9Y,Beauty_Accs,HAIR CARE,Спрей для волос,"[Хороший спрей, запах приятный , Запах приятны...",839,"[хороший спрей, запах приятный, запах приятный...",839,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[134, 69, 29, -1, 2, -1, -1, -1, 26, 147, 42, ...","{0: {'texts': ['прикольная штука', 'крутая шту...","{1: ['запах приятный и'], 3: ['сильных аромат ...","[сильных аромат , волосы, приятный аромат, дер...",6
110599,MP002XW1FQM9,Clothes,VESTS & TOPS,Топ спортивный,[Качество хорошее. Но грудь не держит - съезжа...,779,[качество хорошее. но грудь не держит - съезжа...,779,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[-1, 122, 50, -1, -1, 15, 127, 114, 119, 25, 1...","{0: {'texts': ['обычный топ', 'в целом, обычны...","{1: ['эту цену топ'], 3: ['красивой спинкой, в...","[жару отлично. размер l, мягкий симпатичный,в ...",6
38960,MP002XU0CYA2,Beauty_Accs,HAIR CARE,Шампунь,"[Не понравился, волосы не вымывает , Приятный ...",762,"[не понравился, волосы не вымывает, приятный з...",762,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[-1, 101, 30, -1, 83, 33, 99, -1, 107, 107, 93...","{0: {'texts': ['волосы реально стали лучше', '...","{2: ['слова совсем. волосы после него', 'волос...","[аромат приятный. для моих волос, пробовала. н...",6
118034,RTLABM368801,Beauty_Accs,HAIR STYLING,Расческа для распутывания волос,"[Расческа жесткая, но я и брала для массажа, е...",685,"[расческа жесткая, но я и брала для массажа, е...",685,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[-1, 103, 20, 33, 55, 72, -1, 53, 66, 7, 24, 7...","{0: {'texts': ['купила ! разной жестокости, но...","{0: ['очень нравятся расчески этой 

In [2]:
# ============================================================================
# PROJECT SETUP & MODEL INITIALIZATION
# ============================================================================

PROJECT_DIR = Path().resolve()
INTERIM_DIR = PROJECT_DIR / "data/interim"
FILE_PATH = PROJECT_DIR / "data/raw/lamoda_reviews.csv"

# Инициализация модели для embeddings
print("="*80)
print("INITIALIZING EMBEDDING MODEL")
print("="*80)

# Пробуем загрузить модель из кеша
model_candidates = [
    'paraphrase-multilingual-MiniLM-L12-v2',  # Лучше для русского
    'all-MiniLM-L6-v2',
    'all-mpnet-base-v2',
]

sentence_model = None
USE_TFIDF_FALLBACK = False

for model_name in model_candidates:
    try:
        print(f"Trying to load model: {model_name}...")
        sentence_model = SentenceTransformer(model_name, local_files_only=True)
        print(f"✓ Model '{model_name}' loaded successfully!")
        break
    except Exception as e:
        print(f"✗ {model_name}: Not found in cache")
        continue

if sentence_model is None:
    print("\n⚠ No sentence transformer models found in cache")
    print("Using TF-IDF as fallback for embeddings")
    USE_TFIDF_FALLBACK = True
else:
    print(f"\n✓ Using model: {model_name}")
    print(f"  Embedding dimension: {sentence_model.get_sentence_embedding_dimension()}")
print("="*80)


No sentence-transformers model found with name sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2. Creating a new one with mean pooling.
No sentence-transformers model found with name sentence-transformers/all-MiniLM-L6-v2. Creating a new one with mean pooling.
No sentence-transformers model found with name sentence-transformers/all-mpnet-base-v2. Creating a new one with mean pooling.


INITIALIZING EMBEDDING MODEL
Trying to load model: paraphrase-multilingual-MiniLM-L12-v2...
✗ paraphrase-multilingual-MiniLM-L12-v2: Not found in cache
Trying to load model: all-MiniLM-L6-v2...
✗ all-MiniLM-L6-v2: Not found in cache
Trying to load model: all-mpnet-base-v2...
✗ all-mpnet-base-v2: Not found in cache

⚠ No sentence transformer models found in cache
Using TF-IDF as fallback for embeddings


In [3]:
# ============================================================================
# ШАГ 0: ИСХОДНЫЕ ДАННЫЕ
# ============================================================================
# Объекты: sku_id, category, product_type, reviews

print("="*80)
print("ШАГ 0: ЗАГРУЗКА И ПОДГОТОВКА ДАННЫХ")
print("="*80)

# Загружаем данные
print("Loading data...")
df_raw = pd.read_csv(FILE_PATH)
print(f"Loaded {len(df_raw)} reviews")
print(f"Unique products: {df_raw['product_sku'].nunique()}")
print(f"Categories: {df_raw['good_type'].unique()}")

# Группируем по продуктам и формируем структуру данных
print("\nPreparing data structure...")
products_data = []

for sku, group in df_raw.groupby('product_sku'):
    # Собираем отзывы для продукта
    reviews = group['comment_text'].dropna().tolist()
    
    if len(reviews) < 3:  # Минимум 3 отзыва для анализа
        continue
    
    # Получаем категорию и подтип
    category = group['good_type'].iloc[0] if 'good_type' in group.columns else None
    product_type = group['good_subtype'].iloc[0] if 'good_subtype' in group.columns else None
    product_name = group['name'].iloc[0] if 'name' in group.columns else None
    
    products_data.append({
        'sku': sku,
        'category': category,
        'product_type': product_type,
        'product_name': product_name,
        'reviews': reviews,
        'num_reviews': len(reviews)
    })

# Создаем DataFrame
df_products = pd.DataFrame(products_data)
print(f"✓ Prepared {len(df_products)} products with at least 3 reviews")

# Для тестирования можно использовать подмножество
USE_SUBSET = True
if USE_SUBSET:
    # Берем топ-50 продуктов по количеству отзывов
    df_products = df_products.nlargest(50, 'num_reviews')
    print(f"Using subset: {len(df_products)} products for testing")

print(f"\nExample product structure:")
example = df_products.iloc[0]
print(f"  SKU: {example['sku']}")
print(f"  Category: {example['category']}")
print(f"  Product Type: {example['product_type']}")
print(f"  Reviews: {example['num_reviews']}")
print(f"  Sample review: {example['reviews'][0][:80]}...")

df_products.head()


ШАГ 0: ЗАГРУЗКА И ПОДГОТОВКА ДАННЫХ
Loading data...
Loaded 1774267 reviews
Unique products: 254307
Categories: ['Clothes' 'Accs' 'Shoes' 'Beauty_Accs' 'Home_Accs' 'Bags' 'Toys'
 'Jewellery' nan]

Preparing data structure...
✓ Prepared 160568 products with at least 3 reviews
Using subset: 50 products for testing

Example product structure:
  SKU: RTLACN098001
  Category: Beauty_Accs
  Product Type: MAKE-UP EYES
  Reviews: 1393
  Sample review: Отличные тени, мягкая текстура, хорошо ложатся, не осыпаются. Есть матовые и пер...


,sku,category,product_type,product_name,reviews,num_reviews
119186,RTLACN098001,Beauty_Accs,MAKE-UP EYES,Палетка для глаз,"[Отличные тени, мягкая текстура, хорошо ложатс...",1393
48815,MP002XW0FXPS,Clothes,SOCKS & TIGHTS,Носки 3 пары,"[По качеству хорошие, производитель Узбекистан...",1390
56885,MP002XW0MX30,Beauty_Accs,EYES CARE,Патчи для глаз,"[Хорошие патчи , Самые любимые патчи! , Самые ...",1311
41898,MP002XW00FJB,Beauty_Accs,MAKE-UP BROWS,Гель для бровей,"[Самый лучший гель! , Мне не оч «зашел» гель…(...",1292
109922,MP002XW1FMAK,Clothes,UNDERWEAR,Трусы 4 шт.,"[Дополнение..\nНе очень понравилось, как сидят...",1100


In [4]:
# ============================================================================
# ШАГ 1: ОЧИСТКА ТЕКСТОВ
# ============================================================================
# Вход: reviews: List[str]
# Выход: clean_reviews: List[str]
# Мягкая очистка: приводим к нижнему регистру, убираем мусор, ничего не выкидываем по смыслу

print("="*80)
print("ШАГ 1: ОЧИСТКА ТЕКСТОВ")
print("="*80)

def clean_text_soft(text: str) -> str:
    """
    Мягкая очистка текста (feature cleaning, не feature engineering):
    - Приводим к нижнему регистру
    - Убираем HTML теги
    - Убираем эмодзи (опционально, можно оставить)
    - Убираем лишние символы
    - НЕ удаляем слова по смыслу
    """
    if pd.isna(text) or not text:
        return ""
    
    text = str(text)
    
    # Убираем HTML теги
    text = re.sub(r'<[^>]+>', '', text)
    
    # Убираем множественные пробелы и переносы строк
    text = re.sub(r'\s+', ' ', text)
    
    # Убираем управляющие символы
    text = re.sub(r'[\x00-\x1f\x7f-\x9f]', '', text)
    
    # Приводим к нижнему регистру
    text = text.strip().lower()
    
    return text


# Применяем очистку ко всем отзывам всех продуктов
print("Cleaning reviews...")
df_products['clean_reviews'] = df_products['reviews'].apply(
    lambda reviews: [clean_text_soft(r) for r in reviews if clean_text_soft(r)]
)

# Удаляем продукты без отзывов после очистки
df_products = df_products[df_products['clean_reviews'].apply(len) >= 3].copy()
df_products['num_clean_reviews'] = df_products['clean_reviews'].apply(len)

print(f"✓ Cleaned reviews for {len(df_products)} products")
print(f"  Total reviews before: {df_products['num_reviews'].sum()}")
print(f"  Total reviews after: {df_products['num_clean_reviews'].sum()}")

# Показываем пример
example = df_products.iloc[0]
print(f"\nExample (SKU: {example['sku']}):")
print(f"  Original: {example['reviews'][0][:100]}...")
print(f"  Cleaned:  {example['clean_reviews'][0][:100]}...")

df_products[['sku', 'category', 'product_type', 'num_clean_reviews']].head()

ШАГ 1: ОЧИСТКА ТЕКСТОВ
Cleaning reviews...
✓ Cleaned reviews for 50 products
  Total reviews before: 26153
  Total reviews after: 26153

Example (SKU: RTLACN098001):
  Original: Отличные тени, мягкая текстура, хорошо ложатся, не осыпаются. Есть матовые и перламутровые цвета. От...
  Cleaned:  отличные тени, мягкая текстура, хорошо ложатся, не осыпаются. есть матовые и перламутровые цвета. от...


,sku,category,product_type,num_clean_reviews
119186,RTLACN098001,Beauty_Accs,MAKE-UP EYES,1393
48815,MP002XW0FXPS,Clothes,SOCKS & TIGHTS,1390
56885,MP002XW0MX30,Beauty_Accs,EYES CARE,1311
41898,MP002XW00FJB,Beauty_Accs,MAKE-UP BROWS,1292
109922,MP002XW1FMAK,Clothes,UNDERWEAR,1100


In [5]:
# ============================================================================
# ШАГ 2: ПРЕОБРАЗОВАНИЕ ОТЗЫВОВ В ЧИСЛОВЫЕ ВЕКТОРЫ (EMBEDDINGS)
# ============================================================================
# Вход: clean_reviews: List[str]
# Выход: review_embeddings: ndarray shape (n_reviews, embedding_dim)
# Используем transformer-модель, обученную на русском языке
# Это НЕ bag-of-words, модель учитывает смысл предложения

print("="*80)
print("ШАГ 2: ГЕНЕРАЦИЯ EMBEDDINGS")
print("="*80)

def get_embeddings(texts: List[str], batch_size: int = 32) -> np.ndarray:
    """
    Преобразование текстов в числовые векторы фиксированной длины
    FeatureExtractor(text) → R^embedding_dim
    """
    if USE_TFIDF_FALLBACK:
        print(f"Using TF-IDF embeddings (fallback mode)...")
        vectorizer = TfidfVectorizer(
            max_features=384,
            ngram_range=(1, 2),
            min_df=1,
            max_df=0.95
        )
        embeddings = vectorizer.fit_transform(texts).toarray()
        print(f"✓ Generated TF-IDF embeddings shape: {embeddings.shape}")
        return embeddings
    else:
        if sentence_model is None:
            raise ValueError("Sentence transformer model is not loaded!")
        
        print(f"Generating sentence transformer embeddings...")
        embeddings = sentence_model.encode(
            texts,
            batch_size=batch_size,
            show_progress_bar=True,
            convert_to_numpy=True
        )
        print(f"✓ Generated embeddings shape: {embeddings.shape}")
        return embeddings


# Генерируем embeddings для каждого продукта
print("Processing products...")
all_embeddings = []

for idx, row in df_products.iterrows():
    clean_reviews = row['clean_reviews']
    
    # Получаем embeddings для всех отзывов продукта
    embeddings = get_embeddings(clean_reviews)
    all_embeddings.append(embeddings)

# Сохраняем embeddings в DataFrame
df_products['review_embeddings'] = all_embeddings

print(f"\n✓ Generated embeddings for {len(df_products)} products")
print(f"  Embedding dimension: {df_products['review_embeddings'].iloc[0].shape[1]}")

# Показываем пример
example = df_products.iloc[0]
print(f"\nExample (SKU: {example['sku']}):")
print(f"  Reviews: {len(example['clean_reviews'])}")
print(f"  Embeddings shape: {example['review_embeddings'].shape}")

df_products.head()


ШАГ 2: ГЕНЕРАЦИЯ EMBEDDINGS
Processing products...
Using TF-IDF embeddings (fallback mode)...
✓ Generated TF-IDF embeddings shape: (1393, 384)
Using TF-IDF embeddings (fallback mode)...
✓ Generated TF-IDF embeddings shape: (1390, 384)
Using TF-IDF embeddings (fallback mode)...
✓ Generated TF-IDF embeddings shape: (1311, 384)
Using TF-IDF embeddings (fallback mode)...
✓ Generated TF-IDF embeddings shape: (1292, 384)
Using TF-IDF embeddings (fallback mode)...
✓ Generated TF-IDF embeddings shape: (1100, 384)
Using TF-IDF embeddings (fallback mode)...
✓ Generated TF-IDF embeddings shape: (839, 384)
Using TF-IDF embeddings (fallback mode)...
✓ Generated TF-IDF embeddings shape: (779, 384)
Using TF-IDF embeddings (fallback mode)...
✓ Generated TF-IDF embeddings shape: (762, 384)
Using TF-IDF embeddings (fallback mode)...
✓ Generated TF-IDF embeddings shape: (685, 384)
Using TF-IDF embeddings (fallback mode)...
✓ Generated TF-IDF embeddings shape: (624, 384)
Using TF-IDF embeddings (fallback 

,sku,category,product_type,product_name,reviews,num_reviews,clean_reviews,num_clean_reviews,review_embeddings
119186,RTLACN098001,Beauty_Accs,MAKE-UP EYES,Палетка для глаз,"[Отличные тени, мягкая текстура, хорошо ложатс...",1393,"[отличные тени, мягкая текстура, хорошо ложатс...",1393,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
48815,MP002XW0FXPS,Clothes,SOCKS & TIGHTS,Носки 3 пары,"[По качеству хорошие, производитель Узбекистан...",1390,"[по качеству хорошие, производитель узбекистан...",1390,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
56885,MP002XW0MX30,Beauty_Accs,EYES CARE,Патчи для глаз,"[Хорошие патчи , Самые любимые патчи! , Самые ...",1311,"[хорошие патчи, самые любимые патчи!, самые лю...",1311,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
41898,MP002XW00FJB,Beauty_Accs,MAKE-UP BROWS,Гель для бровей,"[Самый лучший гель! , Мне не оч «зашел» гель…(...",1292,"[самый лучший гель!, мне не оч «зашел» гель…(,...",1292,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
109922,MP002XW1FMAK,Clothes,UNDERWEAR,Трусы 4 шт.,"[Дополнение..\nНе очень понравилось, как сидят...",1100,"[дополнение.. не очень понравилось, как сидят ...",1100,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [6]:
# ============================================================================
# ШАГ 3: КЛАСТЕРИЗАЦИЯ ОТЗЫВОВ ПО СМЫСЛУ
# ============================================================================
# Вход: review_embeddings: ndarray
# Выход: clusters: List[int]  # cluster_id для каждого отзыва
# Применяем кластеризацию БЕЗ заранее заданного числа кластеров
# Цель: сгруппировать отзывы по тому, о чём они (материал, комфорт, доставка, шум)

print("="*80)
print("ШАГ 3: КЛАСТЕРИЗАЦИЯ ОТЗЫВОВ")
print("="*80)

def cluster_reviews(embeddings: np.ndarray, min_cluster_size: int = 2) -> np.ndarray:
    """
    Кластеризация отзывов БЕЗ фиксированного числа кластеров
    Используем HDBSCAN для автоматического определения числа кластеров
    Возвращает массив cluster_id для каждого отзыва (-1 = шум/outlier)
    """
    n_samples = len(embeddings)
    
    if n_samples < min_cluster_size:
        # Если отзывов слишком мало, все в один кластер
        return np.zeros(n_samples, dtype=int)
    
    # Используем HDBSCAN для автоматической кластеризации
    # min_cluster_size - минимальный размер кластера
    # min_samples - минимальное количество соседей для core point
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=min_cluster_size,
        min_samples=1,
        metric='euclidean',
        cluster_selection_method='eom'  # Excess of Mass
    )
    
    cluster_labels = clusterer.fit_predict(embeddings)
    
    # Если все отзывы стали шумом, используем KMeans как fallback
    if np.all(cluster_labels == -1):
        n_clusters = min(3, n_samples // 2)
        if n_clusters > 0:
            kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
            cluster_labels = kmeans.fit_predict(embeddings)
    
    return cluster_labels


# Применяем кластеризацию для каждого продукта
print("Clustering reviews for each product...")
all_clusters = []

for idx, row in df_products.iterrows():
    embeddings = row['review_embeddings']
    clean_reviews = row['clean_reviews']
    
    # Кластеризуем отзывы продукта
    cluster_labels = cluster_reviews(embeddings, min_cluster_size=2)
    
    all_clusters.append(cluster_labels)
    
    # Показываем статистику для первых нескольких продуктов
    if idx < 3:
        unique, counts = np.unique(cluster_labels, return_counts=True)
        print(f"\n  Product {row['sku']}:")
        print(f"    Reviews: {len(clean_reviews)}")
        print(f"    Clusters: {len(unique[unique >= 0])} (noise: {np.sum(cluster_labels == -1)})")
        for cluster_id, count in zip(unique, counts):
            if cluster_id >= 0:
                print(f"      Cluster {cluster_id}: {count} reviews")

# Сохраняем кластеры
df_products['clusters'] = all_clusters

print(f"\n✓ Clustered reviews for {len(df_products)} products")

# Показываем общую статистику
total_clusters = sum(len(np.unique(c[c >= 0])) for c in all_clusters)
total_noise = sum(np.sum(c == -1) for c in all_clusters)
print(f"  Total clusters found: {total_clusters}")
print(f"  Total noise/outliers: {total_noise}")

df_products[['sku', 'category', 'num_clean_reviews']].head()

ШАГ 3: КЛАСТЕРИЗАЦИЯ ОТЗЫВОВ
Clustering reviews for each product...

✓ Clustered reviews for 50 products
  Total clusters found: 4721
  Total noise/outliers: 9634


,sku,category,num_clean_reviews
119186,RTLACN098001,Beauty_Accs,1393
48815,MP002XW0FXPS,Clothes,1390
56885,MP002XW0MX30,Beauty_Accs,1311
41898,MP002XW00FJB,Beauty_Accs,1292
109922,MP002XW1FMAK,Clothes,1100


In [7]:
# ============================================================================
# ШАГ 4: РАБОТА С КЛАСТЕРАМИ (НЕ С ОТЗЫВАМИ)
# ============================================================================
# Объект меняется: теперь основная единица анализа = Cluster
# Cluster = набор отзывов с общим смыслом

print("="*80)
print("ШАГ 4: ГРУППИРОВКА ПО КЛАСТЕРАМ")
print("="*80)

def group_reviews_by_clusters(
    clean_reviews: List[str],
    clusters: np.ndarray,
    embeddings: np.ndarray
) -> Dict[int, Dict]:
    """
    Группировка отзывов по кластерам
    Возвращает словарь {cluster_id: {'texts': [...], 'embeddings': [...]}}
    """
    cluster_groups = {}
    
    for cluster_id in np.unique(clusters):
        if cluster_id == -1:
            # Пропускаем шум
            continue
        
        # Находим индексы отзывов в этом кластере
        cluster_indices = np.where(clusters == cluster_id)[0]
        
        cluster_groups[cluster_id] = {
            'texts': [clean_reviews[i] for i in cluster_indices],
            'embeddings': embeddings[cluster_indices]
        }
    
    return cluster_groups


# Группируем отзывы по кластерам для каждого продукта
print("Grouping reviews by clusters...")
all_cluster_groups = []

for idx, row in df_products.iterrows():
    clean_reviews = row['clean_reviews']
    clusters = row['clusters']
    embeddings = row['review_embeddings']
    
    # Группируем по кластерам
    cluster_groups = group_reviews_by_clusters(clean_reviews, clusters, embeddings)
    all_cluster_groups.append(cluster_groups)
    
    # Показываем примеры для первых продуктов
    if idx < 3:
        print(f"\n  Product {row['sku']} ({row['category']}):")
        for cluster_id, group in cluster_groups.items():
            print(f"    Cluster {cluster_id}: {len(group['texts'])} reviews")
            print(f"      Example: {group['texts'][0][:80]}...")

# Сохраняем группировку
df_products['cluster_groups'] = all_cluster_groups

print(f"\n✓ Grouped reviews into clusters for {len(df_products)} products")

# Статистика
total_clusters = sum(len(groups) for groups in all_cluster_groups)
avg_cluster_size = np.mean([
    len(texts) 
    for groups in all_cluster_groups 
    for cluster_id, group in groups.items() 
    for texts in [group['texts']]
])
print(f"  Total clusters: {total_clusters}")
print(f"  Average cluster size: {avg_cluster_size:.2f} reviews")

df_products[['sku', 'category', 'num_clean_reviews']].head()


ШАГ 4: ГРУППИРОВКА ПО КЛАСТЕРАМ
Grouping reviews by clusters...

✓ Grouped reviews into clusters for 50 products
  Total clusters: 4721
  Average cluster size: 3.50 reviews


,sku,category,num_clean_reviews
119186,RTLACN098001,Beauty_Accs,1393
48815,MP002XW0FXPS,Clothes,1390
56885,MP002XW0MX30,Beauty_Accs,1311
41898,MP002XW00FJB,Beauty_Accs,1292
109922,MP002XW1FMAK,Clothes,1100


In [8]:
# ============================================================================
# ШАГ 5: ИЗВЛЕЧЕНИЕ АСПЕКТОВ ИЗ КЛАСТЕРА
# ============================================================================
# Вход: texts_in_cluster: List[str]
# Выход: raw_aspects: List[str]
# Аспект = формулировка свойства товара, которое регулярно упоминается в кластере
# Ищем слова/фразы, которые чаще всего и лучше всего описывают кластер

print("="*80)
print("ШАГ 5: ИЗВЛЕЧЕНИЕ АСПЕКТОВ ИЗ КЛАСТЕРОВ")
print("="*80)

def extract_aspects_from_cluster(
    cluster_texts: List[str],
    min_word_freq: int = 2,
    max_aspects: int = 3
) -> List[str]:
    """
    Извлечение аспектов из кластера
    Находим top-features (слова/фразы), которые отличают этот кластер от других
    
    Технически:
    - Считаем статистику по словам и фразам только внутри кластера
    - Выбираем наиболее частые и информативные
    """
    if len(cluster_texts) == 0:
        return []
    
    # Объединяем все тексты кластера
    all_text = ' '.join(cluster_texts)
    
    # Разбиваем на слова
    words = re.findall(r'\b[а-яёa-z]{3,}\b', all_text.lower())
    
    if len(words) == 0:
        return []
    
    # Подсчитываем частоту слов
    word_counts = Counter(words)
    
    # Фильтруем слишком частые (стоп-слова) и слишком редкие
    # Берем слова, которые встречаются достаточно часто, но не в каждом отзыве
    filtered_words = {
        word: count 
        for word, count in word_counts.items() 
        if count >= min_word_freq and count < len(cluster_texts) * 0.9
    }
    
    # Сортируем по частоте и берем топ
    top_words = sorted(filtered_words.items(), key=lambda x: x[1], reverse=True)[:max_aspects]
    
    # Формируем аспекты из топ-слов
    aspects = []
    for word, count in top_words:
        # Ищем предложения, содержащие это слово
        for text in cluster_texts:
            if word in text.lower():
                # Берем короткую фразу вокруг слова
                words_in_text = text.split()
                if word in [w.lower() for w in words_in_text]:
                    # Берем до 5 слов вокруг ключевого слова
                    word_idx = [i for i, w in enumerate(words_in_text) if word in w.lower()]
                    if word_idx:
                        start = max(0, word_idx[0] - 2)
                        end = min(len(words_in_text), word_idx[0] + 3)
                        phrase = ' '.join(words_in_text[start:end])
                        if len(phrase) > 10 and phrase not in aspects:
                            aspects.append(phrase)
                            break
        
        if len(aspects) >= max_aspects:
            break
    
    # Если не нашли фразы, возвращаем просто топ-слова
    if len(aspects) == 0:
        aspects = [word for word, _ in top_words[:max_aspects]]
    
    return aspects[:max_aspects]


# Извлекаем аспекты для каждого кластера каждого продукта
print("Extracting aspects from clusters...")
all_product_aspects = []

for idx, row in df_products.iterrows():
    cluster_groups = row['cluster_groups']
    
    product_aspects = {}
    
    for cluster_id, group in cluster_groups.items():
        cluster_texts = group['texts']
        
        # Извлекаем аспекты из кластера
        aspects = extract_aspects_from_cluster(cluster_texts, min_word_freq=2, max_aspects=3)
        
        if aspects:
            product_aspects[cluster_id] = aspects
    
    all_product_aspects.append(product_aspects)
    
    # Показываем примеры для первых продуктов
    if idx < 3 and product_aspects:
        print(f"\n  Product {row['sku']} ({row['category']}):")
        for cluster_id, aspects in product_aspects.items():
            print(f"    Cluster {cluster_id}: {aspects}")

# Сохраняем аспекты
df_products['raw_aspects'] = all_product_aspects

print(f"\n✓ Extracted aspects for {len(df_products)} products")

# Статистика
total_aspects = sum(len(aspects) for aspects in all_product_aspects)
print(f"  Total aspects extracted: {total_aspects}")
print(f"  Average aspects per product: {total_aspects / len(df_products):.2f}")

df_products.head()


ШАГ 5: ИЗВЛЕЧЕНИЕ АСПЕКТОВ ИЗ КЛАСТЕРОВ
Extracting aspects from clusters...

✓ Extracted aspects for 50 products
  Total aspects extracted: 1358
  Average aspects per product: 27.16


,sku,category,product_type,product_name,reviews,num_reviews,clean_reviews,num_clean_reviews,review_embeddings,clusters,cluster_groups,raw_aspects
119186,RTLACN098001,Beauty_Accs,MAKE-UP EYES,Палетка для глаз,"[Отличные тени, мягкая текстура, хорошо ложатс...",1393,"[отличные тени, мягкая текстура, хорошо ложатс...",1393,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[-1, -1, 37, -1, 117, 147, 32, 30, -1, 171, -1...","{0: {'texts': ['покупаю не первый раз!', 'перв...","{0: ['раз заказала тени, все было', 'заказала ..."
48815,MP002XW0FXPS,Clothes,SOCKS & TIGHTS,Носки 3 пары,"[По качеству хорошие, производитель Узбекистан...",1390,"[по качеству хорошие, производитель узбекистан...",1390,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[21, 3, -1, 176, 226, 127, 164, -1, 118, 91, -...","{0: {'texts': ['ожидала, что ткань будет плотн...","{0: ['ожидала, что ткань будет', 'что ткань бу..."
56885,MP002XW0MX30,Beauty_Accs,EYES CARE,Патчи для глаз,"[Хорошие патчи , Самые любимые патчи! , Самые ...",1311,"[хорошие патчи, самые любимые патчи!, самые лю...",1311,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[188, 137, 39, -1, 149, 184, -1, 13, 114, 73, ...","{0: {'texts': ['хорошие патчи, только носить и...","{1: ['пожалуй лучшие патчи на', 'пожалуй лучши..."
41898,MP002XW00FJB,Beauty_Accs,MAKE-UP BROWS,Гель для бровей,"[Самый лучший гель! , Мне не оч «зашел» гель…(...",1292,"[самый лучший гель!, мне не оч «зашел» гель…(,...",1292,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[195, 33, 40, 163, 174, 21, -1, -1, -1, -1, 11...","{0: {'texts': ['много геля на щеточке, из-за э...","{3: ['понравился гель фиксирует идеально .', '..."
109922,MP002XW1FMAK,Clothes,UNDERWEAR,Трусы 4 шт.,"[Дополнение..\nНе очень понравилось, как сидят...",1100,"[дополнение.. не очень понравилось, как сидят ...",1100,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[83, 37, -1, -1, 203, 197, 157, 116, 87, 129, ...","{0: {'texts': ['скручиваются', 'звезду сняла ч...","{2: ['цены и качества', 'соотношение цены и'],..."


In [9]:
# ============================================================================
# ШАГ 6: ФИЛЬТРАЦИЯ АСПЕКТОВ С УЧЕТОМ КАТЕГОРИИ И ТИПА ТОВАРА
# ============================================================================
# Вход: raw_aspects + category + product_type
# Выход: filtered_aspects: List[str]
# Бизнес-логика: фильтруем запрещенные аспекты, приоритизируем важные

print("="*80)
print("ШАГ 6: ФИЛЬТРАЦИЯ АСПЕКТОВ (CATEGORY-AWARE)")
print("="*80)

# Словари запрещенных и приоритетных аспектов для разных категорий
FORBIDDEN_ASPECTS = {
    'Clothes': ['доставка', 'упаковка'],  # Не релевантно для одежды
    'Shoes': ['доставка', 'упаковка'],
    'Accs': ['доставка', 'упаковка'],
    'Beauty_Accs': ['размер'],  # Размер не важен для косметики
}

PRIORITY_ASPECTS = {
    'Clothes': ['размер', 'материал', 'качество', 'комфорт'],
    'Shoes': ['размер', 'комфорт', 'качество'],
    'Accs': ['качество', 'материал', 'внешний вид'],
    'Beauty_Accs': ['качество', 'эффект', 'аромат'],
}

def filter_aspects(
    raw_aspects: Dict[int, List[str]],
    category: Optional[str],
    product_type: Optional[str]
) -> List[str]:
    """
    Фильтрация и приоритизация аспектов с учетом категории и типа товара
    """
    # Собираем все аспекты в один список
    all_aspects = []
    for cluster_id, aspects in raw_aspects.items():
        all_aspects.extend(aspects)
    
    if len(all_aspects) == 0:
        return []
    
    # Фильтруем запрещенные аспекты
    forbidden = FORBIDDEN_ASPECTS.get(category, [])
    filtered = [
        aspect for aspect in all_aspects
        if not any(forbidden_word in aspect.lower() for forbidden_word in forbidden)
    ]
    
    # Если после фильтрации ничего не осталось, возвращаем исходные
    if len(filtered) == 0:
        filtered = all_aspects
    
    # Приоритизируем важные аспекты
    priority = PRIORITY_ASPECTS.get(category, [])
    priority_aspects = []
    other_aspects = []
    
    for aspect in filtered:
        is_priority = any(priority_word in aspect.lower() for priority_word in priority)
        if is_priority:
            priority_aspects.append(aspect)
        else:
            other_aspects.append(aspect)
    
    # Сортируем: сначала приоритетные, потом остальные
    result = priority_aspects + other_aspects
    
    # Ограничиваем количество (максимум 6 аспектов)
    return result[:6]


# Применяем фильтрацию для каждого продукта
print("Filtering aspects by category and product type...")
all_filtered_aspects = []

for idx, row in df_products.iterrows():
    raw_aspects = row['raw_aspects']
    category = row['category']
    product_type = row['product_type']
    
    # Фильтруем аспекты
    filtered = filter_aspects(raw_aspects, category, product_type)
    all_filtered_aspects.append(filtered)
    
    # Показываем примеры для первых продуктов
    if idx < 3:
        print(f"\n  Product {row['sku']} ({category}):")
        print(f"    Raw aspects: {sum(len(a) for a in raw_aspects.values())}")
        print(f"    Filtered aspects ({len(filtered)}): {filtered}")

# Сохраняем отфильтрованные аспекты
df_products['filtered_aspects'] = all_filtered_aspects
df_products['num_aspects'] = [len(a) for a in all_filtered_aspects]

print(f"\n✓ Filtered aspects for {len(df_products)} products")

# Статистика
total_aspects = sum(len(a) for a in all_filtered_aspects)
print(f"  Total filtered aspects: {total_aspects}")
print(f"  Average aspects per product: {total_aspects / len(df_products):.2f}")

df_products[['sku', 'category', 'product_type', 'num_aspects', 'filtered_aspects']].head()


ШАГ 6: ФИЛЬТРАЦИЯ АСПЕКТОВ (CATEGORY-AWARE)
Filtering aspects by category and product type...

✓ Filtered aspects for 50 products
  Total filtered aspects: 300
  Average aspects per product: 6.00


,sku,category,product_type,num_aspects,filtered_aspects
119186,RTLACN098001,Beauty_Accs,MAKE-UP EYES,6,"[супер качество по хорошей, отличное качество...."
48815,MP002XW0FXPS,Clothes,SOCKS & TIGHTS,6,"[, на вид качество хорошее, качество не очень,..."
56885,MP002XW0MX30,Beauty_Accs,EYES CARE,6,"[эффекта «вау» не, хоть какой-то эффект, , осо..."
41898,MP002XW00FJB,Beauty_Accs,MAKE-UP BROWS,6,"[эффект ламинирования держится в, эффект ламин..."
109922,MP002XW1FMAK,Clothes,UNDERWEAR,6,"[совсем поняла что с размерным, свой 42-44 взя..."


In [10]:
# ============================================================================
# РЕЗУЛЬТАТЫ И ВИЗУАЛИЗАЦИЯ
# ============================================================================

print("="*80)
print("РЕЗУЛЬТАТЫ ПАЙПЛАЙНА")
print("="*80)

# Формируем финальный DataFrame с результатами
results_df = df_products[[
    'sku', 'product_name', 'category', 'product_type', 
    'num_reviews', 'num_aspects', 'filtered_aspects'
]].copy()

# Переименовываем для удобства
results_df.columns = ['product_sku', 'product_name', 'category', 'product_type', 
                      'num_reviews', 'num_aspects', 'aspects']

print(f"\n✓ Final results for {len(results_df)} products")
print(f"\nStatistics:")
print(f"  Products with aspects: {(results_df['num_aspects'] > 0).sum()}")
print(f"  Average aspects per product: {results_df['num_aspects'].mean():.2f}")
print(f"  Min aspects: {results_df['num_aspects'].min()}")
print(f"  Max aspects: {results_df['num_aspects'].max()}")

# Показываем примеры по категориям
print("\n" + "="*80)
print("EXAMPLES BY CATEGORY")
print("="*80)

for category in results_df['category'].dropna().unique()[:3]:
    cat_products = results_df[results_df['category'] == category].head(3)
    print(f"\n{category}:")
    print("-" * 80)
    for _, row in cat_products.iterrows():
        print(f"\n  Product: {row['product_name']} (SKU: {row['product_sku']})")
        print(f"  Reviews: {row['num_reviews']}, Aspects: {row['num_aspects']}")
        if row['aspects']:
            print(f"  Aspects:")
            for i, aspect in enumerate(row['aspects'], 1):
                print(f"    {i}. {aspect}")

results_df.head(10)


РЕЗУЛЬТАТЫ ПАЙПЛАЙНА

✓ Final results for 50 products

Statistics:
  Products with aspects: 50
  Average aspects per product: 6.00
  Min aspects: 6
  Max aspects: 6

EXAMPLES BY CATEGORY

Beauty_Accs:
--------------------------------------------------------------------------------

  Product: Палетка для глаз (SKU: RTLACN098001)
  Reviews: 1393, Aspects: 6
  Aspects:
    1. супер качество по хорошей
    2. отличное качество. макияж
    3. качество теней отличное
    4. очень хорошее качество, все
    5. отличная палетка,хорошее качество
    6. качество, все цвета носибельные, шимер

  Product: Патчи для глаз (SKU: MP002XW0MX30)
  Reviews: 1311, Aspects: 6
  Aspects:
    1. эффекта «вау» не
    2. хоть какой-то эффект
    3. , особого эффекта нет
    4. не увидела эффекта
    5. не увидела вау эффекта
    6. эффективные, результат есть

  Product: Гель для бровей (SKU: MP002XW00FJB)
  Reviews: 1292, Aspects: 6
  Aspects:
    1. эффект ламинирования держится в
    2. эффект ламинирования

,product_sku,product_name,category,product_type,num_reviews,num_aspects,aspects
119186,RTLACN098001,Палетка для глаз,Beauty_Accs,MAKE-UP EYES,1393,6,"[супер качество по хорошей, отличное качество...."
48815,MP002XW0FXPS,Носки 3 пары,Clothes,SOCKS & TIGHTS,1390,6,"[, на вид качество хорошее, качество не очень,..."
56885,MP002XW0MX30,Патчи для глаз,Beauty_Accs,EYES CARE,1311,6,"[эффекта «вау» не, хоть какой-то эффект, , осо..."
41898,MP002XW00FJB,Гель для бровей,Beauty_Accs,MAKE-UP BROWS,1292,6,"[эффект ламинирования держится в, эффект ламин..."
109922,MP002XW1FMAK,Трусы 4 шт.,Clothes,UNDERWEAR,1100,6,"[совсем поняла что с размерным, свой 42-44 взя..."
37498,MP002XU08J9Y,Спрей для волос,Beauty_Accs,HAIR CARE,839,6,"[сильных аромат , волосы, приятный аромат, дер..."
110599,MP002XW1FQM9,Топ спортивный,Clothes,VESTS & TOPS,779,6,"[жару отлично. размер l, мягкий симпатичный,в ..."
38960,MP002XU0CYA2,Шампунь,Beauty_Accs,HAIR CARE,762,6,"[аромат приятный. для моих волос, пробовала. н..."
118034,RTLABM368801,Расческа для распутывания волос,Beauty_Accs,HAIR STYLING,685,6,"[супер расчёска, качество, расчёска, качество ..."
77577,MP002XW0Z3CQ,Боди,Clothes,UNDERWEAR,624,6,"[не подобрала размер на мой, не понравился тов..."


In [11]:
# ============================================================================
# ЭКСПОРТ РЕЗУЛЬТАТОВ
# ============================================================================

print("="*80)
print("ЭКСПОРТ РЕЗУЛЬТАТОВ")
print("="*80)

# Подготавливаем данные для экспорта
export_df = results_df.copy()
export_df['aspects'] = export_df['aspects'].apply(
    lambda x: ' | '.join(x) if isinstance(x, list) else ''
)

# Сохраняем в CSV
output_csv = INTERIM_DIR / "product_aspects.csv"
export_df.to_csv(output_csv, index=False, encoding='utf-8')
print(f"✓ Results saved to CSV: {output_csv}")

# Сохраняем в JSON (с полными списками аспектов)
output_json = INTERIM_DIR / "product_aspects.json"
results_df.to_json(output_json, orient='records', force_ascii=False, indent=2)
print(f"✓ Results saved to JSON: {output_json}")

# Финальная статистика
print("\n" + "="*80)
print("ФИНАЛЬНАЯ СТАТИСТИКА")
print("="*80)
print(f"Total products processed: {len(results_df)}")
print(f"Products with aspects: {(results_df['num_aspects'] > 0).sum()}")
print(f"Average aspects per product: {results_df['num_aspects'].mean():.2f}")
print(f"\nAspects distribution:")
print(results_df['num_aspects'].value_counts().sort_index())
print(f"\nCategories distribution:")
print(results_df['category'].value_counts())

print("\n" + "="*80)
print("ПАЙПЛАЙН ЗАВЕРШЕН")
print("="*80)


ЭКСПОРТ РЕЗУЛЬТАТОВ
✓ Results saved to CSV: /Users/a.danyarov/Library/CloudStorage/GoogleDrive-abdaniarov@gmail.com/Мой диск/02 Education/01 CU master/lamoda-bootcamp/data/interim/product_aspects.csv
✓ Results saved to JSON: /Users/a.danyarov/Library/CloudStorage/GoogleDrive-abdaniarov@gmail.com/Мой диск/02 Education/01 CU master/lamoda-bootcamp/data/interim/product_aspects.json

ФИНАЛЬНАЯ СТАТИСТИКА
Total products processed: 50
Products with aspects: 50
Average aspects per product: 6.00

Aspects distribution:
num_aspects
6    50
Name: count, dtype: int64

Categories distribution:
category
Beauty_Accs    26
Clothes        20
Shoes           2
Home_Accs       1
Bags            1
Name: count, dtype: int64

ПАЙПЛАЙН ЗАВЕРШЕН


In [14]:
# ============================================================================
# ПАЙПЛАЙН ЗАВЕРШЕН
# ============================================================================
# 
# Структура пайплайна:
#   0. Исходные данные: sku, category, product_type, reviews
#   1. Очистка текстов (мягкие правила)
#   2. Преобразование в числовые векторы (embeddings)
#   3. Кластеризация отзывов по смыслу (без фиксированного числа кластеров)
#   4. Работа с кластерами (группировка отзывов)
#   5. Извлечение аспектов из кластера (поиск частых слов/фраз)
#   6. Фильтрация аспектов с учетом category и product_type
#
# Результаты сохранены в:
#   - data/interim/product_aspects.csv
#   - data/interim/product_aspects.json


In [ ]:
# ============================================================================
# VISUALIZATION: EXAMINE RESULTS
# ============================================================================

# Показываем примеры результатов для разных категорий
print("Examples by category:\n")

for category in results_df['category'].unique()[:3]:
    if pd.isna(category):
        continue
    cat_products = results_df[results_df['category'] == category].head(3)
    print(f"\n{category}:")
    print("-" * 80)
    for _, row in cat_products.iterrows():
        print(f"\n  SKU: {row['product_sku']}")
        print(f"  Name: {row['product_name']}")
        print(f"  Reviews: {row['num_reviews']}")
        print(f"  Tags ({row['tags_count']}):")
        for tag in row['tags']:
            print(f"    - {tag}")


(1774267, 6)

In [ ]:
# Эта ячейка удалена - старый код
